# Multi-Agent Orchestration

![Agent with scientific knowledge tool](../../../static/agent-multi.png)

The final lab in this module demonstrates how to build a sophisticated multi-agent research system that addresses key challenges in multi-agent architectures, particularly the context window management and "game of telephone" problems identified in Anthropic's multi-agent research system blog.

The lab implements a three-agent hierarchical system with clear separation of concerns:

1. Lead Agent (Orchestrator)
    - Role: Planning, coordination, and file management
    - Responsibilities: Decomposes research questions, creates outlines, delegates tasks, and manages the overall workflow
    - Tools: Research agent delegation, report generation tool, file editor
    - Key Feature: Does NOT perform research activities directly, maintaining focus on orchestration
2. Research Agent (Specialist)
    - Role: Scientific literature investigation
    - Responsibilities: Searches PMC databases, gathers evidence using PaperQA2's advanced RAG techniques
    - Tools: PMC search tool, evidence gathering with DynamoDB storage
    - Output: Concise research summaries with evidence IDs for citation tracking
3. Writing Agent (Specialist)
    - Role: Technical report generation
    - Responsibilities: Creates properly cited reports using Anthropic's Claude Citations API
    - Input: Research question and evidence IDs from DynamoDB
    - Output: Professional reports with inline citations and reference lists

Using multiple agents gives use additional LLM context to accomplish the research goal. It also allows us to precisely configure the system prompt, model, and tools for the task at hand.

To address the "game of telephone" challenge, each research agent saves evidence records to a DynamoDb database and only passes the record IDs and a summary of the content back to the lead agent. The lead agent syntheiszes this information and passes the evidence ID values to the writing agent, which queries the DynamoDb table for the details necessary to write a high-quality, cited research report.

## 1. Prerequisites

- Python 3.12 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude 3.7 Sonnet model in Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit paper-qa

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")
warnings.filterwarnings("ignore", module="litellm")
MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"

## 2. Define PMC Research Agent

This agent will be similar to the gather evidence agent we created in notebook 2, but with a change in how the evidence is stored. To avoid the "game of telephone" problem, we are going to deterministically store the gathered evidence in a DynamoDB table. This is a big advantage to building AI agents on AWS - they can leverage any of the 200+ services to store and process data.

Let's test out the updated tool to see how it works.

First, we create a DynamoDB table, with `toolUseId` as the primary key and `pmcid` as a global secondary id. This gives us the flexibility to extract evidence records either by tool use or paper. 

In [ ]:
import boto3
import os

dynamodb = boto3.resource("dynamodb")
DDB_TABLE_NAME = "deep-research-evidence"
os.environ["EVIDENCE_TABLE_NAME"] = DDB_TABLE_NAME

# Check if table exists
try:
    ddb_table = dynamodb.Table(DDB_TABLE_NAME)
    ddb_table.load()
    print(f"Table '{DDB_TABLE_NAME}' already exists")
except:
    # Create table if it doesn't exist
    ddb_table = dynamodb.create_table(
        TableName=DDB_TABLE_NAME,
        KeySchema=[{"AttributeName": "evidence_id", "KeyType": "HASH"}],
        AttributeDefinitions=[
            {"AttributeName": "evidence_id", "AttributeType": "S"},
            {"AttributeName": "source", "AttributeType": "S"},
        ],
        GlobalSecondaryIndexes=[
            {
                "IndexName": "source_index",
                "KeySchema": [{"AttributeName": "source", "KeyType": "HASH"}],
                "Projection": {"ProjectionType": "ALL"},
            }
        ],
        BillingMode="PAY_PER_REQUEST",
    )

    ddb_table.wait_until_exists()

Next, we directly invoke the updated `gather_evidence` tool and pass the db table name as an environment variable.

In [ ]:
from strands import Agent
from search_pmc import search_pmc_tool
from gather_evidence_ddb import gather_evidence_tool

agent = Agent(tools=[gather_evidence_tool], model=MODEL_ID)

# Send a message to the agent
agent.tool.gather_evidence_tool(
    pmc_id="PMC9438179",
    question="How safe and effective are GLP-1 drugs for long term use?",
)

Let's look at the new records in our db table

In [ ]:
for record in ddb_table.scan().get("Items"):
    print("-"*50)
    print(f"Evidence ID: {record.get('evidence_id')}")
    print(f"Question: {record.get('question')}")
    print(f"Answer: {record.get('answer')}")
    print(f"Source: {record.get('source')}")
    print("Context:")
    for i, item in enumerate(record.get('context'), start=1):
        print(f"  {i}: {item}")

We can also query for a records from a specific PMC ID

In [ ]:
from boto3.dynamodb.conditions import Key

response = ddb_table.query(
    IndexName="source_index", KeyConditionExpression=Key("source").eq('PMC9438179')
)

for record in ddb_table.scan().get("Items"):
    print("-"*50)
    print(f"Evidence ID: {record.get('evidence_id')}")
    print(f"Question: {record.get('question')}")
    print(f"Answer: {record.get('answer')}")
    print(f"Source: {record.get('source')}")
    print("Context:")
    for i, item in enumerate(record.get('context'), start=1):
        print(f"  {i}: {item}")

Let's incorporate our tool into a new `pmc_research_agent` definition

In [ ]:
from strands import Agent
from search_pmc import search_pmc_tool
from gather_evidence_ddb import gather_evidence_tool

QUERY = "How safe and effective are GLP-1 drugs for long term use?"

SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

1. Use search_pmc_tool to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 2 years"[dp] for recent work.
2. Identify the PMC IDs of the most relevant papers, then submit each ID and the query to the gather_evidence_tool.
3. Generate a concise answer to the question based on the most relevant evidence, followed by a list of the associated `evidence_id` values.
"""


# Initialize your agent
pmc_research_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[search_pmc_tool, gather_evidence_tool],
    model=MODEL_ID,
)

# Send a message to the agent
response = pmc_research_agent(QUERY)

Now let's view the updated records in our evidence table again

In [ ]:
for record in ddb_table.scan().get("Items"):
    print("-"*50)
    print(f"Evidence ID: {record.get('evidence_id')}")
    print(f"Question: {record.get('question')}")
    print(f"Answer: {record.get('answer')}")
    print(f"Source: {record.get('source')}")
    print("Context:")
    for i, item in enumerate(record.get('context'), start=1):
        print(f"  {i}: {item}")

We'll use our `pmc_research_agent` again later in this notebook

## 3. Define Technical Writing Agent

Next, we'll build our writing agent. This agent will take in a question and evidence references and use the Anthropic Claude citations API to create a cited report. Let's test how this works with the Amazon Bedrock InvokeModel API 

Ref: https://docs.claude.com/en/docs/build-with-claude/citations

### 3.1. Explore Anthropic Claude Citations API

Here is an example of how to use the Claude citations API through a Bedrock InvokeModel call

In [ ]:
import boto3
import json

bedrock_client = boto3.client("bedrock-runtime")

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "Based on the available context, specific recommendations for managing gastrointestinal adverse events during GLP-1 receptor agonist (GLP-1 RA) therapy include several key strategies. Management emphasizes comprehensive patient education, initiating treatment with low starting doses, and implementing gradual dose titration to minimize adverse events (Gorgojo2023 chunk 2). Dietary adjustments and ongoing monitoring are also essential components of the management approach (Gorgojo2023 chunk 2).\n\nRegarding clinical experience with long-term safety, GI adverse events occur in 40-85% of patients receiving GLP-1 RAs, but these events are typically mild and transient in nature (Gorgojo2023 chunk 2). The adverse events generally resolve following dose escalation, indicating that patients can adapt to therapy over time (Gorgojo2023 chunk 2). \n\nLong-term safety data demonstrate a favorable profile, with most events classified as non-serious and low rates of permanent treatment discontinuation (Gorgojo2023 chunk 2). This clinical experience supports the overall safety profile of GLP-1 RAs for long-term use (Gorgojo2023 chunk 2). The combination of appropriate management strategies and the generally mild, self-limiting nature of GI adverse events allows most patients to continue therapy successfully over extended periods.",
                    },
                    "title": "PMC9821052",
                    "citations": {"enabled": True},
                },
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "Long-term GLP-1 receptor agonist therapy demonstrates sustained efficacy in maintaining glycemic control and promoting weight loss (Zheng2024 chunk 55). However, therapeutic effectiveness may plateau over extended treatment periods, with studies indicating this limitation is associated with increased orbitofrontal reward activation, as observed with liraglutide (Zheng2024 chunk 55).\n\nThe adverse event profile is predominantly gastrointestinal, with nausea and vomiting representing the most common side effects (Zheng2024 chunk 55). Serious but rare adverse events include pancreatitis and gallbladder disease (Zheng2024 chunk 55).\n\nSeveral absolute contraindications exist for long-term GLP-1 receptor agonist use. Patients with a personal or family history of medullary thyroid carcinoma should not receive these agents (Zheng2024 chunk 55). Multiple endocrine neoplasia type 2 (MEN 2) also represents a contraindication (Zheng2024 chunk 55). Additionally, severe gastrointestinal disorders preclude the use of these medications (Zheng2024 chunk 55).\n\nThe safety and efficacy profile supports long-term use in appropriate patients, though careful patient selection is essential given the specific contraindications, particularly those related to thyroid malignancy risk and pre-existing gastrointestinal pathology.",
                    },
                    "title": "PMC9821052",
                    "citations": {"enabled": True},
                },
                {
                    "type": "text",
                    "text": "What is the long-term safety profile and effectiveness of GLP-1 receptor agonists? What are the main adverse events and contraindications for long-term use?",
                },
            ],
        }
    ],
    "max_tokens": 1024,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

for content in response_body.get("content"):
    print(content)

We can now format the response into a citated output

In [ ]:
def print_cited_response(response_content: dict) -> None:
    citations = []
    for content_item in response_content:
        print(content_item.get("text"), end="")
        for citation in content_item.get("citations", []):
            title = citation.get("document_title")
            if title not in citations:
                citations.append(title)
            print(f" ({citations.index(title)+1})", end="")

    print("\n")
    print("## References")
    for i, title in enumerate(citations, start=1):
        print(f"{i}. https://www.ncbi.nlm.nih.gov/pmc/articles/{title}")
    return None


print_cited_response(response_body.get("content"))

### 3.2. Create generate_report tool

We've provided a Strands tool definition that incorporates this code in `generate_report.py'. Let's test it with the example records from earlier in this notebook.

In [ ]:
records = [record for record in ddb_table.scan().get("Items")[:2]]
question = records[0]["question"]
evidence = [record.get("evidence_id") for record in records]

print(question)
print(evidence)

In [ ]:
from strands import Agent
from generate_report import generate_report_tool

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
agent = Agent(tools=[generate_report_tool], model=MODEL_ID)

result = agent.tool.generate_report_tool(
    prompt=f"Please write a concise report that answers the question, {question}",
    evidence_ids=evidence,
)

print(result.get("content")[0]["text"])

This "agent" can be used by our lead agent to generate cited, well-written final reports.

## 4. Create Lead Agent

Now that we've defined all of our subagents and tools, we're ready to create our lead agent. This agent will be responsible for planning, updating files, and assigning tasks but NOT to any of the research activities. This separation of concerns allows us to opimize the token usage and simplify the agent definitions.

In [ ]:
from pmc_research_agent import pmc_research_agent
from strands import Agent, tool

@tool
def research_agent(prompt: str) -> str:
    """
    AI agent for researching scientific questions using articles from PubMed Central (PMC).

    You may delegate research tasks to this agent by providing clear text instructions in the prompt.

    Args:
        prompt: Scientific question to research using articles from PMC

    Returns:
        Concise answer to the question based on the most relevant evidence, followed by a list of the associated `evidence_id` values for citation analysis.
    """
    return pmc_research_agent(prompt)

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands_tools import editor
from pmc_research_agent import pmc_research_agent
from generate_report import generate_report_tool
from lead_config import SYSTEM_PROMPT, MODEL_ID

import os

model = BedrockModel(
    model_id=MODEL_ID,
    max_tokens=10000,
    cache_prompt="default",
    temperature=1,
    additional_request_fields={
        "anthropic_beta": ["interleaved-thinking-2025-05-14"],
        "reasoning_config": {
            "type": "enabled",
            "budget_tokens": 3000,
        },
    },
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

lead_agent = Agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[research_agent, generate_report_tool, editor],
)

response = lead_agent(
    "How safe and effective are GLP-1 drugs for long term use? Please limit your output report to only 3 sections"
)
response = lead_agent("I approve the outline. Please proceed.")

response.metrics.accumulated_usage

## 5. Deploy to Amazon Bedrock AgentCore Runtime

Let's look at the new agent definition

In [ ]:
%pycat lead_agent.py

In [ ]:
import boto3
from bedrock_agentcore_starter_toolkit import Runtime

ssm = boto3.client('ssm')

agentcore_runtime = Runtime()
agentcore_runtime.configure(
    agent_name="lead_agent",
    auto_create_ecr=True,
    execution_role=ssm.get_parameter(Name='/deep-research-workshop/agentcore-runtime-role-arn')['Parameter']['Value'],
    entrypoint="lead_agent.py",
    memory_mode="NO_MEMORY",
    requirements_file="requirements.txt",
)

In [ ]:
agentcore_runtime.launch(auto_update_on_conflict=True)

In [ ]:
import uuid
session_id = str(uuid.uuid4())

In [ ]:
from invoke_agentcore import invoke_agentcore
invoke_agentcore(
    agent_runtime_name="lead_agent",
    prompt="How safe and effective are GLP-1 drugs for long term use?",
    session_id = session_id
)

In [ ]:
from invoke_agentcore import invoke_agentcore
invoke_agentcore(
    agent_runtime_name="pmc_deep_research_agent",
    prompt="Looks good! Please continue with section 1.",
    session_id = session_id
)

## 6. (Optional) Interact with agent using AgentCore Chat

Follow these steps to open an interactive chat session with your new agent.

1. Open a command line terminal in your notebook environment.
2. Navigate to the project root folder (where `pyproject.toml` is located).
3. Run `pip install .` to install the workshop tools including the chat CLI.
4. Run `agentcore-chat` to launch the CLI.
5. Select the `lead_agent` by typing its name or index in the terminal and press Enter.
6. Ask your question at the `You:` prompt and press Enter.


## 7. (Optional) Clean Up

Run the next notebook cell to delete the AgentCore runtime environment.

In [ ]:
import boto3

agentcore_client = boto3.client("bedrock-agentcore-control")
agent_status = agentcore_runtime.status()

agentcore_client.delete_agent_runtime(agentRuntimeId=agent_status.config.agent_id)